# Nor operator modeling and characterization
In this notebook we will construct a genetic network to model a Nor operator, a device that is repressed by either of two inputs, upload the simulated data to Flapjack, and then show how to characterize the operator based on this data. The GeneticNetwork will use two Receiver operators to drive the input repressors, and a Nor operator to produce the output based on these inputs.

NOTE: In order to run this notebook, and characterize the Nor operator, you must first run Receiver1.ipynb and Receiver2.ipynb to generate data for the Receivers used in the network.

## Import required packages

In [ ]:
from loica import *
import matplotlib.pyplot as plt
import numpy as np
import getpass

## Make a connection to Flapjack
Note here you should specify which instance of Flapjack you will use, whether it is local or the public instance for example.

In [ ]:
from flapjack import *
#fj = Flapjack(url_base='flapjack.rudge-lab.org:8000')
fj = Flapjack(url_base='localhost:8000')
fj.log_in(username=input('Flapjack username: '), password=getpass.getpass('Password: '))

## Get or create Flapjack objects
To associate with the components of the genetic network and the simulated data with Flapjack we need the Ids of the appropriate objects. Note that if the objects already exist you will be prompted and can simply hit return to use the existing objects.

In [ ]:
receiver1_vector = fj.get('vector', name='receiver1')
receiver2_vector = fj.get('vector', name='receiver2')

In [ ]:
study = fj.create('study', name='Loica testing', description='Test study for demonstrating Loica')

In [ ]:
dna = fj.create('dna', name='nor')
vector = fj.create('vector', name='nor', dnas=dna.id)

In [ ]:
sfp = fj.create('signal', name='SFP', color='green', description='Simulated fluorescent protein')

## Create the network with measurable reporter
First we create a GeneticNetwork object and associate it with a Flapjack Vector (collection of DNA). The connection to Flapjack is optional, but we will use it here to upload data and characterize our components.

In [ ]:
network = GeneticNetwork(vector=vector.id[0])

In [ ]:
reporter = Reporter(name='SFP', color='green', degradation_rate=0, init_concentration=0, signal_id=sfp.id[0])

In [ ]:
network.add_reporter(reporter)

## Create and add the Receiver operators
The receiver operator responds to a signal $s$ to produce an output expression rate $\phi(s)$ modeled as follows:

\begin{equation}
    \phi(s)
    =
    \frac
    {
        a + b (\frac{s}{K})^n
    }
    {
        1 + (\frac{s}{K})^n
    }
\end{equation}

Here we must create two Supplement objects to represent the signals, in this case modeling acyl-homoserine lactones (AHLs). The Receivers drive the input repressors, which then are the inputs to the Nor operator.

In [ ]:
ahl1 = Supplement(name='AHL1')
repressor1 = Regulator('LacI')
rec1 = Receiver(input=ahl1, output=repressor1, a=0, b=100, K=1, n=2)

ahl2 = Supplement(name='AHL2')
repressor2 = Regulator('TetR')
rec2 = Receiver(input=ahl2, output=repressor2, a=0, b=100, K=1, n=2)

network.add_operators([rec1,rec2])
network.add_regulators([repressor1,repressor2])

## Create and add the Nor operator
The Nor operator represents a device which can be repressed by either of two repressors $r_1$ and $r_2$, and is modeled as follows, where $\phi(r_1, r_2)$ is the output expression rate:

\begin{equation}
    \phi(r_1, r_2)
    =
    \frac{
        \alpha_0 
        + 
        \alpha_1 (\frac{r1}{K_1})^{n_1} 
        + 
        \alpha_2 (\frac{r2}{K_2})^{n_2}
        +
        \alpha_3 (\frac{r1 r2}{K_1 K_2})^{n_1 + n_2}
    }
    {
        1 
        + 
        (\frac{r1}{K_1})^{n_1}
        + 
        (\frac{r2}{K_2})^{n_2}
        +
        (\frac{r1 r2}{K_1 K_2})^{n_1 + n_2}
    }
\end{equation}


In [ ]:
nor = Nor(input=[repressor1, repressor2], output=reporter, alpha=[1,0,0,0], K=[100,1], n=[4,2])

In [ ]:
network.add_operator(nor)

## Draw the GeneticNetwork as a graph
We can now make a visual representation of our GeneticNetwork to check it is wired up correctly.

In [ ]:
plt.figure(figsize=(3,3), dpi=150)
network.draw()

## Simulate the GeneticNetwork
In order to simulate the GeneticNetwork behaviour we need to specify the growth conditions in which it will operate. To do this we create a SimulatedMetabolism object which specifies growth functions.

In [ ]:
def growth_rate(t):
    return gompertz_growth_rate(t, 0.05, 1, 1, 1)

def biomass(t):
    return gompertz(t, 0.05, 1, 1, 1)
    
metab = SimulatedMetabolism(biomass, growth_rate)

media = fj.create('media', name='loica', description='Simulated loica media')
strain = fj.create('strain', name='loica', description='Loica test strain')

Now we can create Samples that contain our GeneticNetwork driven by the SimulatedMetabolism. We also need to specify the Media and Strain, in order to link to the Flapjack data model. To test the inverter behaviour we must also add the signals (ahls) at a range of concentrations.

In [ ]:
# Create list of samples    
samples = []
for conc1 in np.append(0, np.logspace(-2, 2, 12)):
    for conc2 in np.append(0, np.logspace(-3, 1, 12)):
        for _ in range(1):
            sample = Sample(genetic_network=network, 
                    metabolism=metab,
                    media=media.id[0],
                    strain=strain.id[0])
            # Add AHL to samples at given concentration
            sample.add_supplement(ahl1, conc1)
            sample.add_supplement(ahl2, conc2)
            samples.append(sample)

Given our Samples, we can now create an Assay which will simulate an experiment containing them. We need to specify the biomass signal in order to link to the Flapjack data model for later upload. Running the assay will simulate the behaviour of the GeneticNetwork.

In [ ]:
biomass_signal = fj.create('signal', name='SOD', description='Simulated OD', color='black')

In [ ]:
assay = Assay(samples, 
              n_measurements=100, 
              interval=0.24,
              name='Loica NOR',
              description='Simulated NOR generated by loica',
              biomass_signal_id=biomass_signal.id[0]
             )
assay.run()

## Upload simulated data to Flapjack

In [ ]:
assay.upload(fj, study.id[0])

Now we can check that the simulation worked by plotting a heatmap using the PyFlapjack package to connect to the Flapjack API. This also allows us to see if we have covered the dynamic range of the Nor operator, in order to correctly characterize it.

In [ ]:
ahl1_id = fj.get('chemical', name='AHL1').id[0]
ahl2_id = fj.get('chemical', name='AHL2').id[0]

In [ ]:
fig = fj.plot(study=study.id,                     
                vector=vector.id,
                signal=sfp.id,
                type='Heatmap',
                analyte1=ahl1_id,
                analyte2=ahl2_id,
                function='Mean Expression',
                biomass_signal=biomass_signal.id[0],
                normalize='None',
                subplots='Signal',
                markers='Vector',
                plot='All data points'
       )
fig

## Characterize the Not operator from the uploaded data

In [ ]:
nor.characterize(fj, 
            receiver1_vector.id,
            receiver2_vector.id, 
            ahl1_id,
            ahl2_id,
            vector.id, 
            media.id, 
            strain.id, 
            sfp.id, 
            biomass_signal.id,
            0
            )

In [ ]:
nor.alpha0, nor.alpha1, nor.alpha2, nor.alpha3

In [ ]:
nor.rep1_K, nor.rep1_n

In [ ]:
nor.rep2_K, nor.rep2_n